In [ ]:
import warnings

warnings.filterwarnings("ignore")

## Linealy Separable Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import CategoricalAccuracy, Loss

from utils import log_training_loss, log_training_results

In [ ]:
from matplotlib import rc

# configurations for dark background
plt.style.use(['dark_background', 'bmh'])

# remove background colour, set figure size
rc('figure', figsize=(16, 8), facecolor='black', max_open_warning=False)
rc('axes', facecolor='none')

In [ ]:
X, y = make_blobs(n_samples=300, n_features=2, 
                           centers=3, center_box=[-5, 5], random_state=42)
y.reshape(-1, 1)

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu)
plt.xlabel("$X1$")
plt.ylabel("$X2$")
plt.colorbar()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Create DataSet

In [ ]:
class ArrayDataset(Dataset):
    
    def __init__(self, data, y=None):
        self.x = data.astype(np.float32)
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        return self.x[idx], self.y[idx]

In [ ]:
train_ds = ArrayDataset(X, y)

In [ ]:
# Now that Dataset is Defined we can create a DataLoader to feed batches
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)

## Build the Model

In [ ]:
n_inputs = 2
n_classes = 3

# Simple Neural Network - No Hidden Layer
model = nn.Sequential(
    nn.Linear(in_features=n_inputs, out_features=n_classes)
)

# Define Optimizer and Loss Function
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

metrics = {
    'accuracy': CategoricalAccuracy(),
    'loss': Loss(criterion)
}
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)

evaluator = create_supervised_evaluator(model,
                                        metrics=metrics,
                                        device=device)

In [ ]:
trainer.add_event_handler(Events.ITERATION_COMPLETED, log_training_loss, 
                           data_loader=train_dl, 
                           log_interval=10)

trainer.add_event_handler(Events.EPOCH_COMPLETED, log_training_results, 
                           evaluator=evaluator,
                           data_loader=train_dl, 
                           metrics=metrics, 
                          is_train=True)

In [ ]:
trainer.run(train_dl, max_epochs=10)